In [ ]:
import sys, os
import numpy as np
import csv
from datetime import datetime, timedelta
# for plotting
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('classic')
%matplotlib inline

In [ ]:
edu_dir = '/Users/jihyun/research/education/'
course_id = 2755
data_dir = os.path.join(edu_dir, 'data',str(course_id))
print data_dir

In [ ]:
first_day = datetime(2016, 9, 15, 0, 0, 0)
days_limit = 90  # total 90 days

# Load csv clickstream data

In [ ]:
def get_cats_from_url(url, depth=1):

    if not 0 < depth < 4:
        raise ('ERROR! depth should be integers between 1 and 3!')

    if url.endswith('/'):
        url = url[:-1]
    url = url.replace('api/v1/', '')

    if len(url.split('/')) < 6:
        return 'homepage'


def load_csv(deidentified_data_dir):
    """
    The de-identified CSV files should have the following columns
        random_id, url_action, created_at, interaction_seconds, ip_address

    """

    print('Loading csv files')
    data = {}
    index = 0
    idx2id = []
    id2idx = {}
    ip2id = {}
    date_format = "%Y-%m-%dT%H:%M:%SZ"

    for file in os.listdir(deidentified_data_dir):
        url = []
        category = []
        action = []
        created_at = []
        interact_secs = []
        remote_ip = []

        if file.endswith('csv'):
            csv_file = os.path.join(deidentified_data_dir, file)
            csv_reader = csv.reader(open(csv_file, 'r'))
            next(csv_reader, None)
            for line in csv_reader:
                id = line[0]
                ip = line[-1]
                url.append(line[1])
                category.append(get_cats_from_url(line[1]))
                action.append(line[2])
                # you need to subtract 7 hours to get the california time
                created_at.append(datetime.strptime(line[3], date_format) - timedelta(0, 25200))
                interact_secs.append(line[4])
                remote_ip.append(ip)
                if ip2id.get(ip, None) is None:
                    ip2id[ip] = []
                if int(id) not in ip2id[ip]:
                    ip2id[ip].append(int(id))

            id = int(id)
            data[id] = {"url": url[::-1], "category": category[::-1], "action": action[::-1],
                        "created_at": created_at[::-1],
                        "interact_secs": interact_secs[::-1], "remote_ip": remote_ip[::-1]}
            id2idx[id] = index
            idx2id.append(id)
            index += 1
    n_students = index
    print('Finished Loading')
    return data, n_students, id2idx, idx2id, ip2id

In [ ]:
deid_data_dir = os.path.join(data_dir, 'deidentified') # path to the deidentified click data
csv_data, n_students, id2idx, idx2id, ip2id = load_csv(deid_data_dir)

In [ ]:
# data checking
print csv_data.keys()
print csv_data[211456]

# Get number of clicks per day matrix

In [ ]:
def get_num_clicks_per_day(student_data, days_limit, first_day, type='all'):
    """
    Parameters
    ----------
    student_data : dict
        dictionary for each student. (One entry of canvas_data.) It should have 'created_at' entry.

    Returns
    -------
        np.array
        numpy array with length 'days_limit'
        Histogram (counts) of the student as a function of time.

    """
    hist_array = np.zeros(days_limit, dtype=np.int32)
    if type == 'all':  # Default
        for time in student_data['created_at']:
            delta = time - first_day
            if delta.days < days_limit:
                hist_array[delta.days] += 1
    else:
        for i, time in enumerate(student_data['created_at']):
            cat = get_cats_from_url(student_data['url'][i], depth=1)
            if cat == type:
                delta = time - first_day
                if delta.days < days_limit:
                    hist_array[delta.days] += 1
    return hist_array


def get_num_clicks_per_day_mat(csvdata, n_students, days_limit, first_day, idx2id, type='all'):
    """
    Get (num_student X num_days) matrix
    where each row is the number of click events per day for each student.
    Returns
    -------
        np.array

    """
    clicks_per_day_mat = np.zeros((n_students, days_limit), dtype=np.int32)
    for idx, random_id in enumerate(idx2id):
        student = csvdata[random_id]
        clicks_per_day_mat[idx] = get_num_clicks_per_day(student, days_limit, first_day, type)
    return clicks_per_day_mat

In [ ]:
click_mat = get_num_clicks_per_day_mat(csv_data, n_students, days_limit, first_day, idx2id)
print click_mat

In [ ]:
plt.bar(range(days_limit), np.mean(click_mat, axis=0))

# Load the quiz_url description file

In [137]:
qname2closedate05 = {}
qname2closedate70 = {}
qname2qidx = {}
qidx2qname = []

quiz_url_file = os.path.join(data_dir, 'quiz_url_descriptions.csv')
with open(quiz_url_file, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    header = reader.next()
    for line in reader:
        q_num = int(line[0])
        q_name = line[1].split(":")[0]
        q_pnts = line[2]
        q_open_date05 = datetime.strptime(line[3], "%m/%d/%Y %H:%M:%S")
        q_close_date05 = datetime.strptime(line[4], "%m/%d/%Y %H:%M:%S")
        q_open_date70 = datetime.strptime(line[8], "%m/%d/%Y %H:%M:%S")
        q_close_date70 = datetime.strptime(line[9], "%m/%d/%Y %H:%M:%S")
        qname2closedate05[q_name] = q_close_date05
        qname2closedate70[q_name] = q_close_date70
        qname2qidx[q_name] = q_num-1
        qidx2qname.append(q_name)

In [138]:
qname2closedate05

{'Lesson 1A': datetime.datetime(2016, 9, 26, 10, 0),
 'Lesson 1B': datetime.datetime(2016, 9, 26, 10, 0),
 'Lesson 1D': datetime.datetime(2016, 9, 28, 10, 0),
 'Lesson 2A': datetime.datetime(2016, 9, 30, 10, 0),
 'Lesson 2B': datetime.datetime(2016, 10, 3, 10, 0),
 'Lesson 2C': datetime.datetime(2016, 10, 5, 10, 0),
 'Lesson 2D': datetime.datetime(2016, 10, 7, 10, 0),
 'Lesson 2E': datetime.datetime(2016, 10, 12, 10, 0),
 'Lesson 3A': datetime.datetime(2016, 10, 19, 10, 0),
 'Lesson 3B': datetime.datetime(2016, 10, 21, 10, 0),
 'Lesson 3C': datetime.datetime(2016, 10, 24, 10, 0),
 'Lesson 3D': datetime.datetime(2016, 10, 26, 10, 0),
 'Lesson 3E': datetime.datetime(2016, 10, 28, 10, 0),
 'Lesson 3F': datetime.datetime(2016, 10, 31, 10, 0),
 'Lesson 3G': datetime.datetime(2016, 11, 2, 10, 0),
 'Lesson 4A': datetime.datetime(2016, 11, 9, 10, 0),
 'Lesson 4B': datetime.datetime(2016, 11, 14, 10, 0),
 'Lesson 4C': datetime.datetime(2016, 11, 18, 10, 0),
 'Lesson 4D': datetime.datetime(2016,

In [139]:
qname2closedate70

{'Lesson 1A': datetime.datetime(2016, 9, 26, 14, 0),
 'Lesson 1B': datetime.datetime(2016, 9, 26, 14, 0),
 'Lesson 1D': datetime.datetime(2016, 9, 28, 14, 0),
 'Lesson 2A': datetime.datetime(2016, 9, 30, 14, 0),
 'Lesson 2B': datetime.datetime(2016, 10, 3, 14, 0),
 'Lesson 2C': datetime.datetime(2016, 10, 5, 14, 0),
 'Lesson 2D': datetime.datetime(2016, 10, 7, 14, 0),
 'Lesson 2E': datetime.datetime(2016, 10, 12, 14, 0),
 'Lesson 3A': datetime.datetime(2016, 10, 19, 14, 0),
 'Lesson 3B': datetime.datetime(2016, 10, 21, 14, 0),
 'Lesson 3C': datetime.datetime(2016, 10, 24, 14, 0),
 'Lesson 3D': datetime.datetime(2016, 10, 26, 14, 0),
 'Lesson 3E': datetime.datetime(2016, 10, 28, 14, 0),
 'Lesson 3F': datetime.datetime(2016, 10, 31, 14, 0),
 'Lesson 3G': datetime.datetime(2016, 11, 2, 14, 0),
 'Lesson 4A': datetime.datetime(2016, 11, 9, 14, 0),
 'Lesson 4B': datetime.datetime(2016, 11, 14, 14, 0),
 'Lesson 4C': datetime.datetime(2016, 11, 18, 14, 0),
 'Lesson 4D': datetime.datetime(2016,

In [91]:
qname2qidx

{'Lesson 1A': 0,
 'Lesson 1B': 1,
 'Lesson 1D': 2,
 'Lesson 2A': 3,
 'Lesson 2B': 4,
 'Lesson 2C': 5,
 'Lesson 2D': 6,
 'Lesson 2E': 7,
 'Lesson 3A': 8,
 'Lesson 3B': 9,
 'Lesson 3C': 10,
 'Lesson 3D': 11,
 'Lesson 3E': 12,
 'Lesson 3F': 13,
 'Lesson 3G': 14,
 'Lesson 4A': 15,
 'Lesson 4B': 16,
 'Lesson 4C': 18,
 'Lesson 4D': 17,
 'Lesson 4E': 19,
 'Lesson 4F': 20,
 'Lesson 4G': 21}

# Load the quiz data 

In [127]:
cid2qsubs = {} # cid to quiz submissions.
cid2section = {}
quiz_data_dir = os.path.join(data_dir, 'Quiz Data')
for fname in os.listdir(quiz_data_dir):
    qname = fname.split("- ")[0]
    filepath = os.path.join(quiz_data_dir, fname)
    
    with open(filepath, 'r') as f:
        reader = csv.reader(f)
        header = reader.next()
        for line in reader:
            cid = int(line[1])
            section = int(line[5].split('-')[-1])
            tstr = line[6]
            submit_time = datetime.strptime(tstr, "%Y-%m-%d %H:%M:%S UTC") - timedelta(hours=7) # to pst time
            attempt = int(line[7])-1 # to use as an index, start with 0
            
            # mapping
            cid2section[cid]= section
            
            if cid not in cid2qsubs:
                cid2qsubs[cid] = {}
            if qname not in cid2qsubs[cid]:
                cid2qsubs[cid][qname] = [None] * 5
                
            cid2qsubs[cid][qname][attempt] = submit_time
            if attempt >= 3:
                print qname, cid, line[0], "- attempt:", attempt+1 

Lesson 4D 6849 Jaymesha Janell Carter - attempt: 4
Lesson 4D 201667 Jamilla Yasmin Abugazia - attempt: 4
Lesson 4D 164841 Sofia Crisheil Cruz Dela Cruz - attempt: 4
Lesson 4D 22212 Alexis Gu Meza Quinonez - attempt: 4
Lesson 4D 205790 Nicolas Hong - attempt: 4
Lesson 4D 204082 Jenny Xie Ng - attempt: 5
Lesson 4D 204082 Jenny Xie Ng - attempt: 4
Lesson 4D 204066 Xilu Zhang - attempt: 5
Lesson 4D 204066 Xilu Zhang - attempt: 4
Lesson 4G 6849 Jaymesha Janell Carter - attempt: 4


In [114]:
# sample output for a single student with id 6849
cid2qsubs[204066]

{'Lesson 1A': [datetime.datetime(2016, 9, 24, 18, 26, 46),
  datetime.datetime(2016, 9, 24, 18, 27, 32),
  None,
  None,
  None],
 'Lesson 1B': [datetime.datetime(2016, 9, 24, 18, 38, 7),
  datetime.datetime(2016, 9, 24, 18, 39, 21),
  None,
  None,
  None],
 'Lesson 1D': [datetime.datetime(2016, 9, 26, 22, 44, 33),
  datetime.datetime(2016, 9, 26, 22, 46, 30),
  datetime.datetime(2016, 9, 26, 22, 52, 46),
  None,
  None],
 'Lesson 2A': [datetime.datetime(2016, 9, 28, 22, 16, 13),
  None,
  None,
  None,
  None],
 'Lesson 2B': [datetime.datetime(2016, 9, 30, 15, 41, 32),
  None,
  None,
  None,
  None],
 'Lesson 2C': [datetime.datetime(2016, 10, 3, 18, 35, 26),
  None,
  None,
  None,
  None],
 'Lesson 2D': [datetime.datetime(2016, 10, 5, 17, 47, 14),
  datetime.datetime(2016, 10, 5, 17, 47, 57),
  None,
  None,
  None],
 'Lesson 2E': [datetime.datetime(2016, 10, 7, 19, 48, 52),
  datetime.datetime(2016, 10, 7, 19, 50, 17),
  None,
  None,
  None],
 'Lesson 3A': [datetime.datetime(2016

In [117]:
cid2qsubs[204066]['Lesson 4D']

[datetime.datetime(2016, 11, 16, 19, 57, 25),
 datetime.datetime(2016, 11, 16, 22, 0, 54),
 datetime.datetime(2016, 11, 16, 22, 1, 27),
 datetime.datetime(2016, 11, 16, 22, 1, 52),
 datetime.datetime(2016, 11, 16, 22, 2, 14)]

# Create a quiz submission matrix to save

In [192]:
# number of colums per quiz
# Change this to 3 and run the cells after this to have 3 cols each
ncols = 5

In [193]:
Nquiz = len(qname2qidx)
subdates = []
for cid in cid2qsubs.keys():
    row = [None] * (Nquiz*ncols + 1)
    for qname in cid2qsubs[cid].keys():
        qidx = qname2qidx[qname]
        row[0] = cid
        col_st_idx = qidx*ncols + 1
        col_end_idx = qidx*ncols + ncols + 1
        row[col_st_idx:col_end_idx] = cid2qsubs[cid][qname][:ncols]
    subdates.append(row)

In [194]:
# sample output for row 0
subdates[0]

[202240,
 datetime.datetime(2016, 9, 24, 17, 25, 16),
 None,
 None,
 None,
 None,
 datetime.datetime(2016, 9, 24, 17, 39, 17),
 datetime.datetime(2016, 9, 24, 17, 41, 3),
 datetime.datetime(2016, 9, 24, 17, 42, 54),
 None,
 None,
 datetime.datetime(2016, 9, 27, 15, 3, 53),
 datetime.datetime(2016, 9, 27, 15, 4, 33),
 None,
 None,
 None,
 datetime.datetime(2016, 9, 29, 17, 5, 34),
 datetime.datetime(2016, 9, 29, 17, 6, 38),
 None,
 None,
 None,
 datetime.datetime(2016, 10, 2, 4, 39, 57),
 None,
 None,
 None,
 None,
 datetime.datetime(2016, 10, 3, 20, 39, 4),
 datetime.datetime(2016, 10, 3, 20, 39, 31),
 None,
 None,
 None,
 datetime.datetime(2016, 10, 6, 2, 31, 34),
 datetime.datetime(2016, 10, 6, 2, 32, 29),
 None,
 None,
 None,
 datetime.datetime(2016, 10, 11, 18, 12, 22),
 None,
 None,
 None,
 None,
 datetime.datetime(2016, 10, 19, 0, 7, 25),
 datetime.datetime(2016, 10, 19, 0, 7, 59),
 None,
 None,
 None,
 datetime.datetime(2016, 10, 20, 21, 48, 52),
 datetime.datetime(2016, 10, 20,

In [195]:
# create header
header = ["canvasid"] + ["quiz"+str(i+1)+"submit"+str(j+1) for i in range(Nquiz) for j in range(ncols)]
print header

['canvasid', 'quiz1submit1', 'quiz1submit2', 'quiz1submit3', 'quiz1submit4', 'quiz1submit5', 'quiz2submit1', 'quiz2submit2', 'quiz2submit3', 'quiz2submit4', 'quiz2submit5', 'quiz3submit1', 'quiz3submit2', 'quiz3submit3', 'quiz3submit4', 'quiz3submit5', 'quiz4submit1', 'quiz4submit2', 'quiz4submit3', 'quiz4submit4', 'quiz4submit5', 'quiz5submit1', 'quiz5submit2', 'quiz5submit3', 'quiz5submit4', 'quiz5submit5', 'quiz6submit1', 'quiz6submit2', 'quiz6submit3', 'quiz6submit4', 'quiz6submit5', 'quiz7submit1', 'quiz7submit2', 'quiz7submit3', 'quiz7submit4', 'quiz7submit5', 'quiz8submit1', 'quiz8submit2', 'quiz8submit3', 'quiz8submit4', 'quiz8submit5', 'quiz9submit1', 'quiz9submit2', 'quiz9submit3', 'quiz9submit4', 'quiz9submit5', 'quiz10submit1', 'quiz10submit2', 'quiz10submit3', 'quiz10submit4', 'quiz10submit5', 'quiz11submit1', 'quiz11submit2', 'quiz11submit3', 'quiz11submit4', 'quiz11submit5', 'quiz12submit1', 'quiz12submit2', 'quiz12submit3', 'quiz12submit4', 'quiz12submit5', 'quiz13submi

In [196]:
# save the data matrix
outfile = "./quiz_submissions.csv"
with open(outfile, 'w') as f:
    writer = csv.writer(f, delimiter=",")
    writer.writerow(header)
    writer.writerows(subdates)

# Time prior to the deadline matrix

In [159]:
delta_prior_due = []

for cid in cid2qsubs.keys():
    row = [None] * (Nquiz + 2)
    sectid = cid2section[cid]
    row[0] = cid
    row[1] = sectid
    
    for qname in cid2qsubs[cid].keys():
        qidx = qname2qidx[qname]
        if sectid == 44170:
            due = qname2closedate70[qname]
        else:  # 44205
            due = qname2closedate05[qname]
        
        last_dt = None
        for dt in cid2qsubs[cid][qname][::-1]:
            if dt != None:
                last_dt = dt
                 
        delta = due - last_dt
        tot_sec = delta.total_seconds()
        hr = tot_sec // 3600
        left = tot_sec % 3600
        minu = left // 60
        sec = left % 60
        delta_str = "%02d:%02d:%02d" % (hr, minu, sec)
   
#         print cid, sectid, qname, delta_str
        row[qidx+2] = delta_str
    
    delta_prior_due.append(row)

In [164]:
delta_prior_due[0]

[202240,
 44170,
 '44:34:44',
 '44:20:43',
 '22:56:07',
 '20:54:26',
 '33:20:03',
 '41:20:56',
 '35:28:26',
 '19:47:38',
 '13:52:35',
 '16:11:08',
 '64:13:50',
 '23:01:49',
 '17:04:08',
 '44:48:57',
 '16:44:16',
 '00:53:48',
 None,
 None,
 '63:17:26',
 '13:22:45',
 None,
 '37:39:27']

In [168]:
## sanity check 

# 202240, 44170
# lesson 4g
# sub : datetime(2016, 11, 27, 0, 20, 33)
# due : datetime(2016, 11, 28, 14, 0)

print (datetime(2016, 11, 28, 14, 0) - datetime(2016, 11, 27, 0, 20, 33) )

1 day, 13:39:27


In [170]:
# create header
header = ["canvasid", "section"] + ["quiz"+str(i+1) for i in range(Nquiz)]
print header

['canvasid', 'section', 'quiz1', 'quiz2', 'quiz3', 'quiz4', 'quiz5', 'quiz6', 'quiz7', 'quiz8', 'quiz9', 'quiz10', 'quiz11', 'quiz12', 'quiz13', 'quiz14', 'quiz15', 'quiz16', 'quiz17', 'quiz18', 'quiz19', 'quiz20', 'quiz21', 'quiz22']


In [172]:
# save the data matrix
outfile = "./quiz_time_prior_due.csv"
with open(outfile, 'w') as f:
    writer = csv.writer(f, delimiter=",")
    writer.writerow(header)
    writer.writerows(delta_prior_due)